In [1]:
from sklearn.datasets import fetch_20newsgroups
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import names
from nltk.stem import PorterStemmer
import numpy as np
import nltk
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from numpy import asarray
from numpy import savetxt

nltk.download('names')
ps = PorterStemmer()
all_names = names.words()
WNL = WordNetLemmatizer()

path_train= "/Users/antonis/Downloads/20news-bydate/20news-bydate-train"
path_test= "/Users/antonis/Downloads/20news-bydate/20news-bydate-test"



def gather(path):
    df = pd.DataFrame()
    for file in os.listdir(path):
        tag = file
        for doc in os.listdir(path+'/'+file):
            docpath = path+'/'+file+'/'+doc
            f = open(docpath, "r",encoding='cp1252')
            content = f.read()
            temp = pd.DataFrame(
                {
                    'content':content,
                    'tag':tag
                },index=[0]
            )
            df = pd.concat([df, temp])



    df.content =df.content.replace(to_replace='From:(.*\n)', value='', regex=True) ##remove from to email
    df.content =df.content.replace(to_replace='lines:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='Subject:(.*\n)', value='', regex=True)
    df.content =df.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]', value=' ', regex=True) #remove punctuation except
    df.content =df.content.replace(to_replace='-', value=' ', regex=True)
    df.content =df.content.replace(to_replace='\s+', value=' ', regex=True)    #remove new line
    df.content =df.content.replace(to_replace='  ', value='', regex=True)                #remove double white space
    df.content =df.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace

    df['content']=[entry.lower() for entry in df['content']]
    return df

df_news_train = gather(path_train)



def clean(data):
    cleaned = defaultdict(list)
    count = 0
    for group in data:
        for words in group.split():
            if words.isalpha() and words not in all_names:
                words = ps.stem(words)
                cleaned[count].append(WNL.lemmatize(words.lower()))
        cleaned[count] = ' '.join(cleaned[count])
        count +=1
    return(list(cleaned.values()))

x_train = clean(df_news_train['content'])


tf = TfidfVectorizer(stop_words='english', max_features=8000,use_idf=True)
tfidf = tf.fit_transform(x_train)

vocab = tf.vocabulary_



[nltk_data] Downloading package names to /Users/antonis/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english',vocabulary=vocab)

count_wm = countvectorizer.fit_transform(x_train)

#using Tfidftransformer and count vectorizer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(count_wm)


# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=countvectorizer.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

# count matrix 
count_vector=countvectorizer.transform(x_train) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = countvectorizer.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[10342] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=['tfidf'],ascending=False)

#using Tfidfvectorizer

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf[0] 

# place tf-idf values in a pandas data frame
df_tfidf = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tf.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

def get_key(val):
    for key, value in tf.vocabulary_.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [3]:
#PART 2 ... TEST


def clean_test(data):
    cleaned = defaultdict(list)
    count = 0
    for words in data.split():
        if words.isalpha() and words not in all_names:
            words = ps.stem(words)
            cleaned[count].append(WNL.lemmatize(words.lower()))
    cleaned[count] = ' '.join(cleaned[count])
    count +=1
    return(list(cleaned.values()))

df_news_test = gather(path_test)

df_news_test['content'] = df_news_test['content'].apply(clean_test)

# x_test = clean(df_news_test['content'])



In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import metrics
from sklearn.metrics import jaccard_score
import numpy



def cosine_similarity_test(document,tfidf):
    testvector = tf.transform(document)
    distances = cosine_similarity(testvector, tfidf)
    prediction = np.argmax(distances, 1)
    predicted = df_news_train['tag'].iloc[prediction]
    return predicted

#df=df_news_test
def classify(df):
    classified = pd.DataFrame()
    doc_count = len(df.index)
    correct = 0
    for row in df.itertuples():
        predicted = cosine_similarity_test(row[1],tfidf)
        temp = pd.DataFrame(
        {
            'document':row[1],
            'class':predicted
        },index=[0])
        classified = pd.concat([classified, temp])
        actual = row[2]
        if actual == predicted[0]:
            correct=correct + 1
    accuracy = correct/doc_count
    percentage = "{:.0%}".format(accuracy)
    print(percentage)
    return classified

# classified_docs = classify(df_news_test)   
    
# def jaccard_similarity_test(document,tfidf):

# testvector = tf.transform(df_news_test['content'].iloc[0])

# distance = euclidean_distances(testvector,tfidf)

# prediction = np.argmax(distance, 1)
# print(prediction)


classified = classify(df_news_test)



64%


In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import metrics
from sklearn.metrics import jaccard_score
import numpy



def cosine_similarity_test(document,tfidf):
    testvector = tf.transform(document)
    distances = euclidean_distances(testvector, tfidf)
    prediction = np.argmax(distances, 1)
    predicted = df_news_train['tag'].iloc[prediction]
    return predicted

#df=df_news_test
def classify(df):
    classified = pd.DataFrame()
    doc_count = len(df.index)
    correct = 0
    for row in df.itertuples():
        predicted = cosine_similarity_test(row[1],tfidf)
        temp = pd.DataFrame(
        {
            'document':row[1],
            'class':predicted
        },index=[0])
        classified = pd.concat([classified, temp])
        actual = row[2]
        if actual == predicted[0]:
            correct=correct + 1
    accuracy = correct/doc_count
    percentage = "{:.0%}".format(accuracy)
    print(percentage)
    return classified

# classified_docs = classify(df_news_test)   
    
# def jaccard_similarity_test(document,tfidf):

testvector = tf.transform(df_news_test['content'].iloc[0])

distance = euclidean_distances(testvector,tfidf)

prediction = np.argmax(distance, 1)
print(prediction)


            

  (0, 1974)	0.08461100813909206
  (0, 4043)	0.02648576450181207
  (0, 30)	0.05000062345558237
  (0, 5342)	0.0387185237363595
  (0, 7569)	0.049829870998868646
  (0, 323)	0.056922108705844986
  (0, 7085)	0.019290752634784514
  (0, 6558)	0.06011884350932519
  (0, 3243)	0.045374092374743616
  (0, 4947)	0.02954481903322534
  (0, 6118)	0.0750476737092194
  (0, 2527)	0.047869992942181895
  (0, 345)	0.06538650079388515
  (0, 3686)	0.06095987137615718
  (0, 2157)	0.0624801305988817
  (0, 4418)	0.06259605333403873
  (0, 1752)	0.11198205669253763
  (0, 3119)	0.05182365837852098
  (0, 4612)	0.062308216525113085
  (0, 5185)	0.06389539434378337
  (0, 7538)	0.034363128666885685
  (0, 1992)	0.08017092382763645
  (0, 5752)	0.09477465241014069
  (0, 5353)	0.06036044722786349
  (0, 466)	0.0673463788505719
  :	:
  (0, 1729)	0.041361114858655346
  (0, 7047)	0.07264038142570269
  (0, 5142)	0.08940604527165598
  (0, 4890)	0.2290849039294512
  (0, 3616)	0.4423452427610115
  (0, 788)	0.06966707274970536
  (0, 

In [1]:
print(tfidf)

NameError: name 'tfidf' is not defined